In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 100

import time
from datetime import datetime, timedelta

In [ ]:
#Start Spark with BigDL support
from pyspark import SparkContext
import bigdl
import bigdl.util.common
sc = SparkContext.getOrCreate(conf=bigdl.util.common.create_spark_conf().setMaster("local[2]")
                              .set("spark.driver.memory","4g"))
bigdl.util.common.init_engine()

In [ ]:
#Increase this to 20 or so for a real run
n_epochs = 1

<!-- requirement: images/VAE.png -->

# Auto-encoders

Autoencoders are neural networks where the number of input and output neurons are the same. If our input neurons represent pixels in an image, the output of our autoencoder will ideally be the input image. Why would we want to create a model that simply reproduces our data? If we restrict the number of neurons in our hidden layers to be less than the number of input or output neurons, we force our model to learn sparse representations of the data. Therefore, autoencoders can be used for image compression and removing noise from images. 

![VAE](images/VAE.png)

An autoencoder consists of two neural networks -- an **encoder** and **decoder**. The encoder takes in high dimensional data and generates low dimensional representations of that data. Then, the decoder takes the low dimensional representations and translates them back into the high dimensional input space. 


### Applications: Dimension reduction, noise removal

The dimension reduction aspect is fairly clear&mdash;we just look at the small, middle layer.  That determines the size of the representation.  Smaller sizes give more compression, but less fidelity in reconstruction.

The noise removal aspect is somewhat less obvious.  During training, the decoder learns how to go from the sparse representation to the full original.  With this limited information, it is working with something akin to eigenvectors or the reduced dimensionality out of PCA.  An analogy is fitting a polynomial to some data&mdash;the fit learns the coefficients as best it can, and the reproduction step always produces a smooth curve.  The noise inherent in the data is lost.

## Making a model

In this tutorial, we will build a simple autoencoder to recreate images in the MNIST data set. We will start off, as usual, loading our data. To make things easier, we'll turn our pixel values from `uint8` to `float` values in the range $[0,1]$.  Our image display utility understands both.

In [ ]:
from bigdl.dataset import mnist
from bigdl.util.common import Sample

mnist_path = "datasets/mnist"
(train_images, train_labels) = mnist.read_data_sets(mnist_path, "train")
(test_images, test_labels) = mnist.read_data_sets(mnist_path, "test")

# We don't need the labels, since we're going to train directly on the pixels
# Reshape images to vectors
mnist_train = sc.parallelize(train_images).map(lambda x: Sample.from_ndarray(x.reshape(28*28)/256.0,x.reshape(28*28)/256.0))
mnist_test = sc.parallelize(test_images).map(lambda x: Sample.from_ndarray(x.reshape(28*28)/256.0,x.reshape(28*28)/256.0))

### Encoder and decoder

We're going to make a fairly small symmetric auto-encoder.  This will consist of an encoder, to take the input image to the reduced dimensionality, and a decoder, to take the dimensionality back to the original.

We will build and train this as a single model, but also create the encoder and decoders of the correct shape.  After training the full model, we'll transfer the appropriate trained weights to the other models.  In principle we should just hook the output of the encoder in to the decoder then train that way, but this is a little complicated in BigDL.

In [ ]:
from bigdl.nn import layer

#The last encoder size is the compressed representation
input_size = 28*28
encoder_size = [256,128]

encode_sizes = [input_size] + encoder_size

model = layer.Sequential()
encoder = layer.Sequential()
for i in range(len(encode_sizes) - 1):
    model.add(layer.Linear(encode_sizes[i],encode_sizes[i+1]))
    model.add(layer.Sigmoid())
    encoder.add(layer.Linear(encode_sizes[i],encode_sizes[i+1]))
    encoder.add(layer.Sigmoid())

decoder = layer.Sequential()
decode_sizes = list(reversed(encode_sizes))
for i in range(len(decode_sizes) - 1):
    model.add(layer.Linear(decode_sizes[i],decode_sizes[i+1]))
    model.add(layer.Sigmoid())
    decoder.add(layer.Linear(decode_sizes[i],decode_sizes[i+1]))
    decoder.add(layer.Sigmoid())

In [ ]:
#And to make sure everything came out the correct shape
print(model.forward(np.ones([7,28*28])).shape)
print(encoder.forward(np.ones([11,28*28])).shape)
print(decoder.forward(np.ones([5,128])).shape)

In [ ]:
#We'll need these later
[x.shape for x in model.get_weights()]

Our loss here is just the difference between the input pixels and the output pixels, so we'll use the mean squared error.  Since we're using a sigmoid layer at the end, we're assured that we have values in the correct range.

The `MSECriterion` thankfully does the right thing, and subtracts our target and output element wise then averages over the (squared) differences.

We will be using the `Adam` optimizer, as we have done a few times already.  As a reminder, we can interpret the learning rate as the size of the step we take down a gradient of our loss function. If the step size is too large, we may never get to the minimum. A large learning rate will manifest itself as noise in our loss curve that never converges to a minimum point. However, if we have a very small step size, our model may take a long time to run. Ideally, we want to take large steps at the start of the training process and small steps towards the end. The [Adam Optimizer](https://arxiv.org/pdf/1412.6980v8.pdf) has a momentum term that prevent this from becoming a problem, and also allows for learning rate decay. 

In [ ]:
from bigdl.nn import criterion 
from bigdl.optim import optimizer

#Using Adam rather than SGD, usually performs better
fitter = optimizer.Optimizer(model=model, training_rdd=mnist_train, criterion=criterion.MSECriterion(), 
                     optim_method=optimizer.Adam(), end_trigger=optimizer.MaxEpoch(n_epochs), 
                                batch_size=60)

#add tracking
now_string = datetime.now().strftime("%Y%m%d-%H%M%S")
trainSummary = optimizer.TrainSummary("./logs", "mnist_ae_{}".format(now_string))
trainSummary.set_summary_trigger("Loss", optimizer.EveryEpoch())
fitter.set_train_summary(trainSummary)

#and checkpointing
#fitter.set_checkpoint(optimizer.EveryEpoch(), './logs/minst_ae_checkpoint_{}'.format(now_string))

In [ ]:
%%time
trained_model = fitter.optimize()

Are we fit?

In [ ]:
summary = trainSummary.read_scalar("Loss")
plt.plot([x[0] for x in summary], [x[1] for x in summary])

Let's see how we did.  We'll compare a few of our images to their reconstructions - and we'll use _test_ data, a separate data set that our network hasn't seen before, to make sure it hasn't just memorized things.

In [ ]:
n_examples = 10
img_shape = (28, 28)

# Applying encode and decode over test set
encode_decode = model.predict(mnist_test).take(n_examples)

# Compare original images with their reconstructions
f, a = plt.subplots(2, n_examples, figsize=(20, 4))
for i in range(n_examples):
    a[0][i].imshow(np.reshape(test_images[i], img_shape))
    a[1][i].imshow(np.reshape(encode_decode[i], img_shape))

## Variational auto-encoder

Variational Autoencoders (VAEs) differ from regular autoencoders, because they not only learn sparse representations of data but also generate new data. Consequently, VAEs are used to [create new images](https://openai.com/blog/generative-models/). For example, we could train a VAE on the MNIST data set and have it create an image of a handwritten "5." 

How do VAEs generate new data? They do so by making smart assumptions about the distributions of these sparse data representations, or **latent vectors**. More generally, they belong to a class of models called generative models, which learn the joint probability distribution between the input ($x$) and output (or latent vectors, $z$). We can then use this information to come up with likely $(x,z)$ pairs. For example, once we learn the distribution corresponding to the sparse representation of a handwritten "5," we can sample from this distribution to form new latent vectors for "5." 

Due to this constraint on the distributions of $z$, VAEs require an additional component in their loss function that penalizes deviations from these distributions.   

We want our network to be accurate, but we also want the latent variables to approximate the posterior distribution. The amount of information lost when approximating $P(z \mid x)$ is called the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback–Leibler_divergence), and we would use it to construct our loss function. To make our lives simple, we usually choose the posterior distribution to be a unit normal and [calculate](http://allisons.org/ll/MML/KL/Normal/) the divergence accordingly. 

Why do we want our latent variables to approximate a certain distribution? We want them to be useful, so we impose this constraint. We can think of this as a form of regularization where we lose some fidelity to ensure we are capturing only important features. In other words, we want to build a model that can generate images and not just memorize them. A nice explanation of choosing latent variables can be found [here](http://kvfrans.com/variational-autoencoders-explained/).

We also want to minimize the loss due to inaccurate pixel values and must therefore have a component of the loss function that penalizes these errors. Our loss function would thus be a combination of a pixel-by-pixel difference and the KL divergence.

We won't actually be doing this here, as they are very difficult and time consuming to train properly.

## Generating new examples

But we can cheat a bit.  We don't know the underlying 128-dimensional distribution of our data, but we can see what happens when we manipulate the compressed representation.  Let's try making a new 4 by making a linear combination of two old 4's.  

First, we'll separate out the encoder from the decoder.  We'll do this by taking our encoder and decoder we created and setting their weights to the appropriate section of weights from our full model.

In [ ]:
model_weights = model.get_weights()
n_weights = len(encoder_size) * 2
# Two weights for each linear set - the W and the b.
encoder.set_weights(model_weights[:n_weights])
decoder.set_weights(model_weights[n_weights:])

In [ ]:
# These are two rather different looking 4's.
first4 = 210
second4 = 4
four1 = test_images[first4].reshape(1,28*28) / 256.0
four2 = test_images[second4].reshape(1,28*28) / 256.0

plt.imshow(four1.reshape(28,28))
plt.grid(False)
plt.show()
plt.imshow(four2.reshape(28,28))
plt.grid(False)
plt.show()

Let's see how the model does reconstructing them before we move forward.

In [ ]:
plt.imshow(model.predict(four1.reshape(1,28*28)).reshape(28,28))
plt.grid(False)
plt.show()

plt.imshow(model.predict(four2.reshape(1,28*28)).reshape(28,28))
plt.grid(False)
plt.show()

Now, we get their encoded representations and linearly interpolate between them.

In [ ]:
code1 = encoder.forward(four1.reshape(1,28*28))
code2 = encoder.forward(four2.reshape(1,28*28))

In [ ]:
new4 = [decoder.forward(code1*p + code2*(1-p)) for p in np.linspace(0,1,n_examples)]
f, a = plt.subplots(1, n_examples, figsize=(20, 4))
for i in range(n_examples):
    a[i].imshow(np.reshape(new4[i], img_shape))

### Exercise: New numbers
Generate a few new numbers as combinations of the old ones (they don't have to be linear combinations of two of them, you could do any size set, but remember that each pixel must be between 0 and 1).

### Exercise: More compression

We have chosen a hidden size of 128, which is about a seventh of the original size, and it does very well.  Try playing with this size and see what happens if you make it smaller.  We tried with 10 and got surprisingly good results.  Why might this be?

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*